In [6]:
import pandas as pd
import pickle

# Carregar os dados
activity_train = pd.read_csv('./activity_train.csv')
activity_test_blanked = pd.read_csv('./activity_test_blanked.csv')
mol_bits = pd.read_pickle('./mol_bits.pkl')

# with open("mol_bits.pkl", "rb") as f:
#     mol_bits = pickle.load(f)

# Visualizar os dados
print("Activity Train Data")
print(activity_train.head())

print("\nActivity Test Blanked Data")
print(activity_test_blanked.head())


Activity Train Data
   O14842   CHEMBL2022243   4
0  O14842   CHEMBL2022244   6
1  O14842   CHEMBL2022245   2
2  O14842   CHEMBL2022246   1
3  O14842   CHEMBL2022247   4
4  O14842   CHEMBL2022248   4

Activity Test Blanked Data
   O14842   CHEMBL2022258  0
0  O14842   CHEMBL2047161  0
1  O14842   CHEMBL2047163  0
2  O14842   CHEMBL2047168  0
3  O14842   CHEMBL2047169  0
4  O14842   CHEMBL2048621  0


In [8]:
activity_train_columns = activity_train.columns
print("Colunas de activity_train:", activity_train_columns)

Colunas de activity_train: Index(['O14842', ' CHEMBL2022243', ' 4'], dtype='object')


In [ ]:
activity_train.fillna(0, inplace=True)
activity_test_blanked.fillna(0, inplace=True)

from sklearn.model_selection import train_test_split

y = activity_train['activity']
X = activity_train.drop(columns=['activity'])

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

print("\nDados prontos para treinamento e validação:")
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

In [ ]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import cross_validate

reader = Reader(rating_scale=(1, 10))
data = Dataset.load_from_df(activity_train[['protein_id', 'molecule_id', 'activity']], reader)

svd = SVD()

cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()
svd.fit(trainset)

activity_test_blanked['predicted_activity'] = activity_test_blanked.apply(
    lambda x: svd.predict(x['protein_id'], x['molecule_id']).est, axis=1)

activity_test_blanked[['protein_id', 'molecule_id', 'predicted_activity']].to_csv('/mnt/data/preds_01.txt', index=False)